## Adding Datetime column to predictive maintenance dataset

In [1]:
# import necessary packages
import pandas as pd
import datetime as dt

In [2]:
# read csv file
df = pd.read_csv("predictive_maintenance.csv")
df

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure
...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,No Failure
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,No Failure
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,No Failure
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,No Failure


In [3]:
# copy dataframe and calculate difference in tool wear rows for difference in minutes
df2 = df.copy()
df2['TWdiff'] =df2['Tool wear [min]'].diff()
dfTW = df2['TWdiff']
dfTW

0       NaN
1       3.0
2       2.0
3       2.0
4       2.0
       ... 
9995    2.0
9996    3.0
9997    5.0
9998    3.0
9999    5.0
Name: TWdiff, Length: 10000, dtype: float64

In [4]:
# drop all negatives values and N/A values, calculate mean
dfTW = dfTW[dfTW >= 0]
dfTW.dropna(inplace=True)
dftwmean = round(dfTW.mean())
dftwmean

3

In [5]:
# fill mean in for all negative and N/A values in tool wear diff variable
dfTWdiff = df2['TWdiff']
dfTWdiff = dfTWdiff.mask(dfTWdiff < 0, dftwmean)
dfTWdiff.fillna(dftwmean, inplace = True)
dfTWdiff

0       3.0
1       3.0
2       2.0
3       2.0
4       2.0
       ... 
9995    2.0
9996    3.0
9997    5.0
9998    3.0
9999    5.0
Name: TWdiff, Length: 10000, dtype: float64

In [6]:
# replace values in dataframe
df2['TWdiff'] = dfTWdiff

In [7]:
# Calculte cumulative summation
TWcumsum = df2['TWdiff'].cumsum()

In [8]:
# Create datetime column from cumulative summation
df2['Datetime'] = dt.datetime(2023,8,1) + pd.TimedeltaIndex(TWcumsum, unit='m')
df2

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type,TWdiff,Datetime
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure,3.0,2023-08-01 00:03:00
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure,3.0,2023-08-01 00:06:00
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure,2.0,2023-08-01 00:08:00
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure,2.0,2023-08-01 00:10:00
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure,2.0,2023-08-01 00:12:00
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,No Failure,2.0,2023-08-19 01:57:00
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,No Failure,3.0,2023-08-19 02:00:00
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,No Failure,5.0,2023-08-19 02:05:00
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,No Failure,3.0,2023-08-19 02:08:00


In [9]:
# Write to csv
df2.to_csv("predictive_maintenance_datetime.csv")